In [1]:
import random
# zmienne wystęujące w algorytmie
L=2            #liczba warstw
mi = 0.1       #współczynnik uczenia
N = [2, 2, 1]  #N_0, N_1, N_2 - liczba neuronów (lub wejsc dla N0) w każdej z warstw

w = [                                    #wartosci wag         
        [],                              #warstwa 0 (brak neuronow)
        [[],[0,0,0],[0,0,0]],            #warstwa 1 (nie ma neuronu 0, neuron 1 i 2 ma 3 wagi
        [[],[0,0,0]]                     #warstwa 2 (nie ma neurony 0, neuron 1 ma 3 wagi)
]

s = [                                   #wartości sum s
        [],                             #warstwa 0 nie posiada sum
        [None, 0, 0],                   #warstwa 1 posiada sume dla neuronu numer 1 i 2 (nie ma neuronu 0 jest dlatego None)
        [None, 0]                       #warstwa 2 posiada sume dla neuronu numwr 1 (nie ma neuronu 0 dlatego jest None)
]

x = [                                   #wartosci wejsc x
        [],                             #warstwa 0 nie posiada wejsc
        [-1,0,0],                       #warstwa 1 posiada 3 wejscia dla kazdego neuronu (x0=+1 oraz x1,x2)
        [-1,0,0]                        #warstwa 2 posiada 3 wejscia dla neuronu nr 1 (x0=+1 oraz x1,x2)
]

y = [                                   #wartosci wyjsciowe neuronow (po wykonaniu funkcji aktywacji f(s))
        [],                             #warstwa 0 nie posiada wyjsc
        [None,0,0],                     #warstwa 1 posiada 2 wyjscia dla dwoch neuronow 1 i 2 (nie ma neuronu 0 więc jest None)
        [None,0]                        #warstwa 1 posiada 1 wyjscie dla neuronu 1 (nie ma neuronu 0 więc jest None)
]

e = [
        [],                             #blad syganlu wyjsciowego
        [None,0,0],
        [None,0]
]

dlt = [                                #wartosc korekty bledu
        [],
        [None,0,0],
        [None,0]
]



In [2]:
#Funkcje wystęujące w algorytmie
import math

#losowanie wag
def losuj_w():
    for k in range(1,L+1):
        for i in range(1,N[k]+1):
            for j in range(0,N[k-1]+1):
                w[k][i][j] = random.random()*2-1

#funkcja aktywacji "sigmoida"
def f(s):
    return 1/(1+math.exp(-s))

#pochodna funkcji aktywacji 
def f_poch(s):
    return f(s)*(1-f(s))

def licz_y(u):
    for k in range(1, L+1):
        for i in range(1,N[k-1]+1):
            if k==1:
                x[k][i] = u[i-1]
            else:
                x[k][i] = y[k-1][i]
        
        for i in range(1,N[k]+1):
            s[k][i] = 0
            for j in range(0,N[k-1]+1):
                s[k][i] += w[k][i][j]*x[k][j]
            y[k][i] = f(s[k][i])
    return y[L][1]

def licz_e(d):
    for k in range(L,1-1,-1):
        for i in range(1,N[k]+1):
            if k==L:
                e[k][i] = d - y[k][i]
            else:
                e[k][i] = 0
                for j in range(1,N[k+1]+1):
                    e[k][i] += dlt[k+1][j]* w[k+1][j][i]
            dlt[k][i] = e[k][i]* f_poch(s[k][i])

In [3]:
losuj_w()
print(w)

u = [ [0,0],   #dane wejsciowe [u1, u2]
      [0,1],
      [1,0],
      [1,1] ]

d = [ 0,       #wzorcowe dane oczekiwane na wyjsciu sieci
      1,
      1,
      0]

[[], [[], [-0.5364742481204297, -0.37345830170271443, 0.7308711692645], [0.8845264815255982, 0.8138166300646803, -0.41196673078544244]], [[], [0.15339891921423, -0.9373837697579182, -0.9755338445538266]]]


In [4]:
import matplotlib.pyplot as plt
wynik = [1,0,0,1]
n=1
M = len(u)
max_n = 10000
for n in range(max_n):
    sum = 0
    for m in range(len(u)):
        uu = u[m]
        dd = d[m]
        wynik[m] = round(licz_y(uu),0)   
        licz_e(dd)
        #for k in range(L,1-1,-1):
        #   for i in range(1,N[k]+1):
        #        for j in range(0,N[k-1]+1):
        for k in range(1,L+1):
            for i in range(1,N[k]+1):
                for j in range(0,N[k-1]+1):
                    w[k][i][j]= w[k][i][j]+ 2*mi*dlt[k][i]*x[k][j]
        sum += math.pow(dd -wynik[m],2)
    #błąd średniokwadratowy
    Q= 1/M * sum
    if(Q < 0.01):
        print("Uczenie zakończono w ", n , "iteracjach")
        break


print(wynik)       

Uczenie zakończono w  2242 iteracjach
[0.0, 1.0, 1.0, 0.0]
